In [1]:
from itertools import product
import sys
from PIL import Image
from os import listdir

import numpy as np
import torch
import torchvision
from torchvision import transforms

from lucent.modelzoo import *
from lucent.misc.io import show
import lucent.optvis.objectives as objectives
import lucent.optvis.param as param
import lucent.optvis.render as render
import lucent.optvis.transform as transform
from lucent.misc.channel_reducer import ChannelReducer
from lucent.misc.io import show

In [2]:
model = torch.hub.load("pytorch/vision:v0.10.0", "googlenet", pretrained=True)
model.eval()

Using cache found in /Users/sturla/.cache/torch/hub/pytorch_vision_v0.10.0


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [3]:
def transform_image(input_image):
    preprocess = transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(
        0
    )  # create a mini-batch as expected by the model
    return input_batch


def predict_category(output, j):
    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    # print(output[j])
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probabilities = torch.nn.functional.softmax(output[j], dim=0)
    # print(probabilities)
    # Read the categories
    with open("data/categories.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    # Show top categories per image
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        print(categories[top5_catid[i]], top5_prob[i].item())


def predict_image(input_batch):
    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to("cuda")
        model.to("cuda")

    with torch.no_grad():
        output = model(input_batch)

    for j in range(output.shape[0]):
        predict_category(output, j)
        print("\n")

In [4]:
input_image = Image.open("data/ILSVRC2015/Data/DET/test/ILSVRC2012_test_00002514.JPEG")
input_batch = transform_image(input_image)
predict_image(input_batch)

277: 'red fox, Vulpes vulpes', 0.6867847442626953
278: 'kit fox, Vulpes macrotis', 0.2572617530822754
280: 'grey fox, gray fox, Urocyon cinereoargenteus', 0.03158718720078468
274: 'dhole, Cuon alpinus', 0.011068567633628845
272: 'coyote, prairie wolf, brush wolf, Canis latrans', 0.0017953580245375633




In [14]:
print(input_batch.shape)

torch.Size([1, 3, 224, 224])


In [5]:
@torch.no_grad()
def get_layer(model, layer, X):
    hook = render.ModuleHook(getattr(model, layer))
    model(X)
    hook.close()
    return hook.features

In [6]:
activation_layer4a = get_layer(model, "inception4a", input_batch)
activation_layer4b = get_layer(model, "inception4b", input_batch)
print(activation_layer4a.shape)
print(activation_layer4b.shape)

torch.Size([1, 512, 14, 14])
torch.Size([1, 512, 14, 14])


In [16]:
print(get_layer(model, "conv1", input_batch).shape)
print(get_layer(model, "maxpool2", input_batch).shape)
print(get_layer(model, "inception3a", input_batch).shape)

torch.Size([1, 64, 112, 112])
torch.Size([1, 192, 28, 28])
torch.Size([1, 256, 28, 28])


In [7]:
act_forward = model.inception4b.forward(activation_layer4a)
print(activation_layer4b == act_forward)

tensor([[[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ...

In [12]:
x_cor, y_cor = 0, 7
sub_sample = activation_layer4a[:, :, x_cor, y_cor][None, None, :]
print(sub_sample.shape)
up_sample = torch.cat([sub_sample] * 14, 0)
up_sample = torch.cat([up_sample] * 14, 1)
up_sample = up_sample.transpose(0, 2).transpose(1, 3)
print(up_sample.shape)

torch.Size([1, 1, 1, 512])
torch.Size([1, 512, 14, 14])


In [18]:
x_cor_new, y_cor_new = 0, 7
upsampled_forward = model.inception4b.forward(up_sample)
ups = upsampled_forward[:, :, x_cor_new, y_cor_new][0].detach().numpy()
reals = activation_layer4b[:, :, x_cor_new, y_cor_new][0].detach().numpy()
num = 0
for index, (up, real) in enumerate(zip(ups, reals)):
    if up != real:
        print(index, up, real)
        num += 1
print(f"Totalt {num} different pairs")

160 0.0 0.38848916
163 0.03733124 0.0
164 1.2267618 0.5919498
165 0.6305436 0.0
166 1.1510599 0.062152743
167 0.42074943 0.0
172 0.0 0.5820501
173 0.22674584 0.0
176 0.0 0.2697202
177 1.0419793 0.9073874
178 0.2761175 0.53378147
179 0.8326369 0.7136029
181 0.12263985 0.4745309
182 0.0 0.7034214
183 0.047688603 0.0
184 0.8309115 0.92085385
185 0.01709491 0.0
188 1.5680778 0.9024359
192 0.0 0.07890089
193 1.8565894 1.588406
194 0.45983 0.032130986
195 0.0 0.10265082
197 0.09772384 0.0
198 0.0 0.35702813
202 0.5700352 1.7355158
203 0.0 0.032739803
204 0.0 0.29623044
207 0.10011524 0.48806107
210 0.5835644 0.0
211 0.16555326 0.49276012
212 1.1704433 1.0098227
213 0.0 0.95695204
215 0.3693323 0.2540036
216 0.17099008 0.0
221 0.97329897 0.85307294
222 1.1701252 0.56040984
223 0.4979653 1.3169535
225 0.0 0.27882162
226 0.12326795 0.0
228 0.0 0.033132106
230 1.0152855 0.51118004
231 0.58856094 0.0
234 2.2756283 1.3592491
235 0.10656176 0.5829259
240 0.49918866 0.0
241 0.6458751 1.0139517
245 0

In [17]:
all_layers = [
    "conv1",
    "maxpool1",
    "conv2",
    "conv3",
    "maxpool2",
    "inception3a",
    "inception3b",
    "maxpool3",
    "inception4a",
    "inception4b",
    "inception4c",
    "inception4d",
    "inception4e",
    "maxpool4",
    "inception5a",
    "inception5b",
    "avgpool",
    "dropout",
    "fc",
]
layers = [
    "inception3a",
    "inception3b",
    "inception4a",
    "inception4b",
    "inception4c",
    "inception4d",
    "inception4e",
    "inception5a",
    "inception5b",
]
# fewer_layers = ["inception3a", "inception3b","inception4a", "inception4b", "inception4c", "inception4d", "inception4e", "inception5a", "inception5b"]

In [18]:
def get_all_layers(model, layers, X):
    hooks = [render.ModuleHook(getattr(model, layer)) for layer in layers]
    model(X)
    for hook in hooks:
        hook.close()
    return [hook.features for hook in hooks]

In [9]:
def get_activations(model, layers, num, path="data/ILSVRC2015/Data/DET/test/"):
    images = []
    for file in listdir(path)[:num]:
        images.append(transform_image(Image.open(path + file)))
    images = torch.cat(images)
    # images = transform_image(images)
    activations = get_all_layers(model, layers, images)
    # activations = [act.mean(dim=[2,3]) for act in activations]
    # activations = [act.max(dim=3)[0].max(dim=2)[0] for act in activations]
    torch.save(biggg, "activations/max_activations.pt")


# predict_image(images)

In [30]:
all_act = get_all_layers(model, layers, input_batch)
for t in all_act:
    print(t.shape)

tot = 0
for t in all_act:
    s = t.shape
    tot += s[1] * s[2] * s[3]
print(tot)
tot2 = 0
for t in all_act:
    s = t.shape
    tot2 += s[1]
print(tot2)
print(tot2 * 250 / tot)

torch.Size([1, 256, 28, 28])
torch.Size([1, 480, 28, 28])
torch.Size([1, 512, 14, 14])
torch.Size([1, 512, 14, 14])
torch.Size([1, 512, 14, 14])
torch.Size([1, 528, 14, 14])
torch.Size([1, 832, 14, 14])
torch.Size([1, 832, 7, 7])
torch.Size([1, 1024, 7, 7])
1235584
5488
1.1104060913705585


In [27]:
torch.save(all_act, "test")

In [15]:
data = torch.load("activations/mean_activations_test.pt")
print([tens.shape for tens in data])

[torch.Size([543, 256]), torch.Size([543, 480]), torch.Size([543, 512]), torch.Size([543, 512]), torch.Size([543, 512]), torch.Size([543, 528]), torch.Size([543, 832]), torch.Size([543, 832]), torch.Size([543, 1024])]


In [23]:
liste = [i for i in range(10)]
list_a, list_b = liste[:5], liste[5:]
print(list_a)
print(list_b)

[0, 1, 2, 3, 4]
[5, 6, 7, 8, 9]


In [37]:
tor = torch.cat([big, big, big])
print(list(map(lambda e: e.shape[3], tor.split(1))))

[7, 7, 7]


In [25]:
test = torch.load("activations/ILSVRC2015/inception5a.pt")

In [26]:
print(test.shape[1])
print(test.count_nonzero() / test.shape[0])

832
tensor(151.0751)


In [33]:
# W = model.inception3a.branch2.conv.weight #[:,:, 0, 0].detach().numpy()
W = model.inception3a.branch2[1].conv.weight  # [:,:, 0, 0].detach().numpy()
W.shape

torch.Size([128, 96, 3, 3])

In [34]:
W[0, 0, :, :]

tensor([[-0.0314, -0.0505,  0.0487],
        [ 0.0578,  0.0576,  0.0633],
        [-0.0079,  0.0038,  0.0350]], grad_fn=<SliceBackward0>)